In [8]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.store.memory import InMemoryStore
load_dotenv()

llm = init_chat_model(
    model = "gpt-4o-mini",
    model_provider='openai',
)

@dataclass
class Context:
    user_name: str

@tool
def remember_preference(runtime: ToolRuntime[Context], preference: str) -> str:
    """
    Tool to extract user preferences and responds.
    """ 
    runtime.store.put(("users",), runtime.context.user_name, {"favorite_topic": preference})
    return f"Got it! I'll remember you like {preference}."

@tool
def recall_preference(runtime: ToolRuntime[Context]) -> str:
    """
    Tool to recall user preferences and respond.
    """
    memory = runtime.store.get(("users",), runtime.context.user_name)
    if memory:
        return f"You like {memory.value['favorite_topic']}."
    return "I don't have any preferences stored yet."

store = InMemoryStore()

agent = create_agent(
    model=llm,
    tools=[remember_preference, recall_preference],
    context_schema=Context,
    store = store,
)


In [10]:
response = agent.invoke({"messages": [{"role": "user", "content": "I like Python"}]}, context=Context(user_name="Narasimhulu"))

In [12]:
response['messages'][-1].content

"I've noted that you like Python programming language! If you have any specific questions or topics about Python you'd like to discuss, feel free to ask!"

In [14]:
agent.invoke({"messages": [{"role": "user", "content": "What do I like?"}]}, context=Context(user_name="Narasimhulu"))['messages'][-1].content

'You like the Python programming language.'